In [4]:
import pandas as pd
import trueskill
from trueskill import Rating 
import numpy as np
import matplotlib.pyplot as pyplot



In [5]:
data = pd.read_csv("/home/kmisiunas/Data/cavendish_foosball_data - Cavendish foosball doubles.csv", index_col=0)


In [7]:
player_names = np.unique( np.concatenate([ 
    data["Blue attack"].unique() ,
    data["Blue defense"].unique() ,
    data["Red attack"].unique() ,
    data["Red defense"].unique() ,
               ]) )


In [8]:
env = trueskill.TrueSkill()

players = { p: Rating() for p in player_names}


In [38]:
# methods for produsing index for Nik

def record_state(time) -> dict:
    out = {"Date": time}
    for p in players:
        out.update( {(p + "_mu") : players[p].mu} )
        out.update( {(p + "_sigma") : players[p].sigma} )
        out.update( {(p + "_score") :players[p].mu - 3*players[p].sigma} )
    
    return out

In [48]:
score_by_time = []

for i in range(len(data)):
    
    if data.iloc[i]["Blue score"] > data.iloc[i]["Red score"]:
        rank = [0,1]
    else:
        rank = [1,0]
        
    p1, p2, p3, p4 = [data["Blue defense"].iloc[i], data["Blue attack"].iloc[i], data["Red defense"].iloc[i], data["Red attack"].iloc[i]] 
    sel = [(players[p1], players[p2]) ,(players[p3], players[p4])] 
    [(players[p1], players[p2]) ,(players[p3], players[p4])]  =  env.rate(sel, ranks=rank)
    
    # populate DataFrame for nik's display methods 
    score_by_time.append( record_state(data.index[i]) )
    
score_by_time = pd.DataFrame(score_by_time).set_index("Date")


In [6]:
print("{0:10}| {1:5} | {2:5} | {3:5}".format("Name", "Mu", "Sigma", "Score"))
print("=================================")

for p in players:
    print("{0:10}| {1:02.2f} | {2:02.2f}  | {3:02.2f}".format(p, players[p].mu, players[p].sigma, players[p].mu - 3*players[p].sigma))

Name      | Mu    | Sigma | Score
Alex      | 29.24 | 1.33  | 25.25
Jeff      | 18.19 | 1.98  | 12.25
Jehangir  | 26.73 | 1.66  | 21.77
Kareem    | 28.68 | 1.22  | 25.02
Karolis   | 24.53 | 1.37  | 20.40
Kurt      | 16.71 | 4.38  | 3.56
Michael   | 23.67 | 1.35  | 19.61
Nik       | 24.29 | 1.44  | 19.97
Ryan      | 20.80 | 5.43  | 4.53
Ulrich    | 22.84 | 1.74  | 17.61
Vlad      | 24.88 | 1.33  | 20.88


In [9]:
res = []
for p in players.keys():
    res.append({'Player': p, 
              'mu':    players[p].mu, 
              'sigma': players[p].sigma, 
              'score': players[p].mu - 3*players[p].sigma})

rating =  pd.DataFrame(res)



In [10]:
rating.sort_values('score',ascending=False)


,Player,mu,score,sigma
0,Alex,29.235359,25.249606,1.328584
3,Kareem,28.679555,25.020568,1.219662
2,Jehangir,26.732929,21.767685,1.655081
10,Vlad,24.877359,20.881805,1.331851
4,Karolis,24.525284,20.403436,1.373949
7,Nik,24.294901,19.969765,1.441712
6,Michael,23.673046,19.610785,1.354087
9,Ulrich,22.841504,17.613108,1.742799
1,Jeff,18.194506,12.250233,1.981424
8,Ryan,20.802562,4.526882,5.425227


---

# Winning probabilities 

In [11]:
import itertools
import math

BETA=4.166

def win_probability(team1, team2):
    delta_mu = sum(r.mu for r in team1) - sum(r.mu for r in team2)
    sum_sigma = sum(r.sigma ** 2 for r in itertools.chain(team1, team2))
    size = len(team1) + len(team2)
    denom = math.sqrt(size * (BETA * BETA) + sum_sigma)
    ts = trueskill.global_env()
    return ts.cdf(delta_mu / denom)

In [12]:
win_probability( 
    team1 = [players["Ulrich"], players["Michael"]] , 
    team2 = [players["Alex"], players["Kareem"]] 
)

0.13409465817080782